In [147]:
from tqdm import tqdm
from transformers import RobertaTokenizer, RobertaConfig, RobertaModel
import os
import pandas as pd
import json
import sys
tqdm.pandas()


def rm_cmts(in_, language):
    if language == 'java':
        try:
            strings = in_.split('\n')
            strings = [i.strip() for i in strings]
            return_text = ''

            multiple = False
            for i in strings:
                comment = False
                if i.startswith('/*'):
                    multiple = True
                    comment = True
                if i.endswith('*/'):
                    multiple = False
                    comment = True
                if multiple == True and i.startswith('*'):
                    comment = True
                if i.startswith('//'):
                    comment = True   
                if comment == False:
                    return_text += i+'\n'
            return return_text
        except:
            return ' '
    elif language == 'py':
        try:
            strings = in_.split('\n')
            strings = [i.rstrip() for i in strings]
            return_text = ''

            multiple = False
            for i in strings:
                comment = False
                if i.startswith('\'\'\''):
                    multiple = True
                    comment = True
                if i.endswith('\'\'\''):
                    multiple = False
                    comment = True
                if multiple == True:
                    comment = True
                if i.startswith('#'):
                    comment = True   
                if comment == False:
                    return_text += i+'\n'
            return return_text
        except:
            return ' '

        
def rm_parse(in_):
    return_text = in_
    try:
        parse(in_)
    except:
        return_text = ' '
        
    return return_text



def clean(text, language):
    text_cmt = rm_cmts(text, language)
    text_cmt_ast = rm_parse(text_cmt)
    text_cmt_ast_spctkn = text_cmt_ast.replace('<img', '').replace('https:', '')
    return text_cmt_ast_spctkn



year = '2017'
language = 'java'

if language == 'java':
    from javalang.parse import parse
elif language == 'py':
    from ast import parse

json_path = '/home/jovyan/Source_Code_Veri/data/GCJ/MS_C2T/gcj'+year+'_'+language
ds_all = pd.read_csv('/home/jovyan/Source_Code_Veri/data/GCJ/gcj'+year+'.csv')

ds_all['language'] = ds_all.apply(lambda row: os.path.splitext(row['file'])[1][1:] , axis=1)
ds = ds_all.loc[ds_all.language==language]
print(ds.shape)
if language == 'py':
    for i in range(ds.shape[0]):
        a = ds.flines.iloc[i]
        try:
            ds.flines.iloc[i] = a.replace('\n ', '\n')
        except:
            pass

ds['flines_clean'] = ds.flines.progress_apply(lambda x: clean(x, language)) 

# ds_codebert = pd.DataFrame(columns=['repo', 'path', 'func_name', 'original_string', 'language', 'code', 'code_tokens', 'docstring', 'docstring_tokens'])
# ds_codebert['repo'] = ds['username']
# ds_codebert['path'] = ds['full_path']
# ds_codebert['func_name'] = ds['task']
# ds_codebert['language'] = language
# ds_codebert['code'] = ds['flines_clean']
# ds_codebert['code_tokens'] = ds['flines_clean'].apply(lambda row: row.split())
# ds_codebert['original_string'] = ' '
# ds_codebert['docstring'] = ' '
# ds_codebert['docstring_tokens'] = ' '

# ds_codebert['sha'] = ' '
# ds_codebert['url'] = ' '
# ds_codebert['partition'] = ' '

# dict_codebert = ds_codebert.to_dict('records')


  0%|          | 10/29018 [00:00<05:03, 95.63it/s]

(29018, 10)


100%|██████████| 29018/29018 [03:38<00:00, 132.80it/s]
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:102: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [155]:
ds_all = pd.read_csv('/home/jovyan/Source_Code_Veri/data/GCJ/gcj'+year+'.csv')
ds_all['language'] = ds_all.apply(lambda row: os.path.splitext(row['file'])[1][1:] , axis=1)

In [ ]:
ds = ds_all.loc[ds_all.language==language]

In [148]:
aa = ds.loc[ds['flines_clean']!=' ']
print(aa.shape)

(27924, 11)


In [156]:
df = pd.read_csv('/home/jovyan/Source_Code_Veri/data/GCJ/gcj2017.csv')

In [157]:
df['language'] = df.apply(lambda row: os.path.splitext(row['file'])[1][1:] , axis=1)

java = df.loc[ds_all.language=='java']
py = df.loc[ds_all.language=='py']

In [159]:
py.shape

(36057, 10)

In [143]:
strings = ds.flines.iloc[13].split('\n')
strings = [i.rstrip() for i in strings]
return_text = ' '

multiple = False
for i in strings:
    comment = False
    if i.startswith('\'\'\''):
        multiple = True
        comment = True
    if i.endswith('\'\'\''):
        multiple = False
        comment = True
    if multiple == True:
        comment = True
    if i.startswith('#'):
        comment = True   
    if comment == False:
        return_text += i+'\n'

In [145]:
return_text

' from copy import deepcopy\nfrom collections import namedtuple\nimport functools\nimport numpy as np\nimport common\nfrom common import read_input, save_cases\n\ndef run(f):\n\ttxt = open(f).read()\n\tf2 = f + \'.out\'\n\toutputs = [calculate(recipe, all_packages)\n\t\t\t   for recipe, all_packages in parse_input(txt)]\n\tsave_cases(f2, outputs)\n\ndef parse_input(txt):\n\tlines = iter([x for x in txt.split(\'\\n\') if x])\n\tT = int(lines.next())\n\tinputs = []\n\tfor line in lines:\n\t\tN, P = line.split()\n\t\tN, P = int(N), int(P)\n\t\trecipe = [int(x) for x in lines.next().split()]\n\n\t\tarr = []\n\t\tfor _ in range(N):\n\t\t\tline = lines.next()\n\t\t\tarr += [sorted(int(x) for x in line.split())]\n\t\tinputs += [(recipe, arr)]\n\treturn inputs\n\n\ndef agree(recipe, packages):\n\tassert len(recipe) == len(packages)\n\tbounds = 0., float("inf")\n\thighs = [np.floor((p / 0.9) / r) for r, p in zip(recipe, packages)]\n\tlows  = [np.ceil((p / 1.1) / r)  for r, p in zip(recipe, pack